1. PCA and SVD (from scratch)

    
    Parsing the ARFF FILE
    


In [3]:
def parse_arff(file_path):
    data_started = False
    attributes = []
    data = []
    nominal_mapping = []
    with open(file_path, 'r') as file:
        for line in file:
            line = line.strip()

            if not line or line.startswith('%'):
                continue  # Skip comments and empty lines

            if line.lower().startswith('@relation'):
                continue  # Skip relation information

            if line.lower().startswith('@attribute'):
                parts = line.split()
                attr_name = parts[1].strip()

                if '{' in line:  # Nominal data
                    values = line[line.index('{') + 1:line.index('}')].split(',')
                    attributes.append((attr_name, 'nominal', values))
                    # Sample attribute information
                    attribute_info = ((attr_name, 'nominal', values))
                    
                    # Extract relevant information
                    attr_name, attr_type, attr_values = attribute_info

                    #Create a mapping dictionary
                    nominal_mapping.append( {value: index for index, value in enumerate(attr_values)} )


                else:  # Numeric data
                    attributes.append((attr_name, 'numeric', 0))

            if line.lower().startswith('@data'):
                data_started = True
                continue

            if data_started:
                data_line = line.split(',')
                
                data.append(line.split(','))

    return attributes, data


file_path = r'C:\JASPER SEM 1 23-24\CS3101N\V4 data\2017 Q1.arff'
attributes, data = parse_arff(file_path)

print("\nData:")

for row in data :
    for i in range ( len (attributes) ):
        attr_name, attr_type, attr_values = attributes[i]
        if attr_type == 'nominal' :
            # Create a mapping dictionary
            nominal_mapping = {value: index for index, value in enumerate(attr_values)}
            row[i] = nominal_mapping.get(row[i])
        elif attr_type == 'numeric' :
            try :
                row[i] = float(row[i])
            except Exception as e:
                # Handle the exception
                row[i] = 1

print (data)


Data:
[[10.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.0], [22.0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 90336.59, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1.0], [27.0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 2, 1, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, -22451.01, 2, 2, 2, 2, 2, 2, 2, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1.0], [73.0, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 3, 2, 3, 3, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 3, 3, 3, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 19337.98, 3, 3, 3, 3, 3, 3, 3, 0, 0, 1, 1, 1, 1, 1

In [30]:
def mean_vector(X):
   """
   Compute the mean vector of the dataset X.
   """
   means = [sum(x) / len(x) for x in zip(*X)]

   # Compute the standard deviation of each feature
   std_devs = [((sum((xi - mean) ** 2 for xi in x) / len(x)) ** 0.5) for x, mean in zip(X, means)]
   
   # Standardize the dataset
   z = [[(xi - mean) / std_dev for xi, mean, std_dev in zip(x, means, std_devs)] for x in X]
   return means, z



def covariance_matrix(X, mean):
    """
    Compute the covariance matrix of the dataset X.
    """
    n = len(X[0])
    cov_matrix = [[0.0 for _ in range(n)] for _ in range(n)]
    i = 0
    col = []
    for x in zip(*X):
        sum = 0
        new_col = [0] * len(x)
        for y in range(len(x)) :
            new_col[y] = x[y] - mean[i]
            sum += ((x[y] - mean[i])**2 )
        col.append(new_col)
        cov_matrix[i][i] = sum / (len(X[0]))
        i+=1
    for y in range(len(col[0])):
        sum += (col[0][y] * col[1][y])
    covar = sum / n
    for i in range(n) :
        for j in range(n) :
            if i != j :
                cov_matrix[i][j] = covar
    return cov_matrix
    
def dot_product(v1, v2):
    return sum(x * y for x, y in zip(v1, v2))

def normalize_vector(v):
    length = (sum(x ** 2 for x in v)) ** 0.5
    return [x / length for x in v]

def matrix_vector_multiply(matrix, vector):
    return [dot_product(row, vector) for row in matrix]

def power_iteration(matrix, num_iterations=100):
    # Initialize a random vector as the initial guess
    eigenvector_guess = [1] * len(matrix)

    for _ in range(num_iterations):
        # Calculate the matrix times the current eigenvector guess
        matrix_times_guess = matrix_vector_multiply(matrix, eigenvector_guess)

        # Normalize the resulting vector
        eigenvector_guess = normalize_vector(matrix_times_guess)

        # Calculate the dominant eigenvalue
        eigenvalue_guess = dot_product(eigenvector_guess, matrix_times_guess)

    return eigenvalue_guess, eigenvector_guess

def matrix_subtract(matrix1, matrix2):
    # Subtract corresponding elements of two matrices
    return [[x - y for x, y in zip(row1, row2)] for row1, row2 in zip(matrix1, matrix2)]

def top_k_eigenpairs(matrix, k):
    eigenpairs = []

    for _ in range(k):
        eigenvalue, eigenvector = power_iteration(matrix)
        eigenpairs.append((eigenvalue, eigenvector))

        # Deflate the matrix to find the next eigenpair
        outer_product = [[v_i * v_j for v_j in eigenvector] for v_i in eigenvector]
        deflate_matrix = matrix_subtract(matrix, [[eigenvalue * y for y in row] for row in outer_product])
        matrix = deflate_matrix
    
    return eigenpairs

def pca(X, k):
    """
    Perform PCA on the dataset X to reduce its dimensionality to k dimensions.
    """
    mean, X = mean_vector(X)
    # print("mean = ", mean, "X=", X)
    cov_matrix = covariance_matrix(X, mean)
    
    print("covariance matrix = ", cov_matrix[0])
    print()
    top_eigenpairs = top_k_eigenpairs(cov_matrix, k)

    # for i, (eigenvalue, eigenvector) in enumerate(top_eigenpairs, start=1):
    #     print(f"Eigenvalue {i}: {eigenvalue}")
    #     print(f"Eigenvector {i}: {eigenvector}")
    #     print()
        # eigenvalues.append(eigenvalue)
        # eigenvectors.append(eigenvector)

    eigenvalues = [eigenvalue for eigenvalue, _ in top_eigenpairs]
    eigenvectors = [eigenvector for _, eigenvector in top_eigenpairs]

    # Sort the eigenvalues and their corresponding eigenvectors
    sorted_indices = sorted(range(len(eigenvalues)), key=lambda i: eigenvalues[i], reverse=True)
    sorted_eigenvalues = [eigenvalues[i] for i in sorted_indices]
    sorted_eigenvectors = [eigenvectors[i] for i in sorted_indices]
    print("Eigen Values = ", sorted_eigenvalues)
    print()
    # print("vectors = ", sorted_eigenvectors)
    # print()
    # Select the top k eigenvectors
    top_k_eigenvectors = [sorted_eigenvectors[i] for i in range(k)]
    
    # Transform the original dataset using the top k eigenvectors
    X_pca = [[sum(x_i * eigenvector_i for x_i, eigenvector_i in zip(x, top_k_eigenvector)) 
              for x, top_k_eigenvector in zip(X, top_k_eigenvectors)] for top_k_eigenvector in top_k_eigenvectors]
    return X_pca

def svd(matrix):
    # Assuming matrix is a list of lists representing a 2D matrix

    # Step 1: Compute A^T * A and A * A^T
    ata = [[sum(a * b for a, b in zip(row_a, col_b)) for col_b in zip(*matrix)] for row_a in matrix]
    aat = [[sum(a * b for a, b in zip(row_a, row_b)) for row_b in matrix] for row_a in matrix]

    # Step 2: Compute eigenvalues and eigenvectors for A^T * A and A * A^T
    eigen_ata = top_k_eigenpairs(ata, len(ata[0]))
    eigen_aat = top_k_eigenpairs(aat, len(aat[0]))
    # print("eigen ata ", eigen_ata)
    eigenvalues_ata = [eigenvalue for eigenvalue, _ in eigen_ata]
    eigenvectors_ata = [eigenvector for _, eigenvector in eigen_ata]

    eigenvalues_aat = [eigenvalue for eigenvalue, _ in eigen_aat]
    eigenvectors_aat = [eigenvector for _, eigenvector in eigen_aat]

    # Step 3: Compute singular values and singular vectors
    # singular_values = [eigenvalue**0.5 for eigenvalue in eigenvalues_ata]

    # Step 4: Construct U, Sigma, and V
    U = eigenvectors_aat  # Left singular vectors
    Sigma = [[0.0] * len(matrix[0]) for _ in range(len(matrix))]  # Diagonal matrix of singular values
    V = 0 #eigenvectors_ata  # Transpose of right singular vectors
    
    # for i in range(len(matrix)):
    #     Sigma[i][i] = singular_values[i]

    return U, Sigma, V
    
ignore, std_data = mean_vector(data)
PCA = pca(data, 2)
print("PCA = ")
for x in PCA :
    print(x)
    print()
# svd(std_data)
U, Sigma, VH = svd(std_data)

# print("U:", U)
# print("Sigma:", Sigma)
# print("V^T:", VH)  # Note: V is returned as VH (transpose of V) in the code

# matrx = [x * 10 for x in matrx]
# print (matrx)

covariance matrix =  [269208.96474016405, 1369.2604041181676, 1369.2604041181676, 1369.2604041181676, 1369.2604041181676, 1369.2604041181676, 1369.2604041181676, 1369.2604041181676, 1369.2604041181676, 1369.2604041181676, 1369.2604041181676, 1369.2604041181676, 1369.2604041181676, 1369.2604041181676, 1369.2604041181676, 1369.2604041181676, 1369.2604041181676, 1369.2604041181676, 1369.2604041181676, 1369.2604041181676, 1369.2604041181676, 1369.2604041181676, 1369.2604041181676, 1369.2604041181676, 1369.2604041181676, 1369.2604041181676, 1369.2604041181676, 1369.2604041181676, 1369.2604041181676, 1369.2604041181676, 1369.2604041181676, 1369.2604041181676, 1369.2604041181676, 1369.2604041181676, 1369.2604041181676, 1369.2604041181676, 1369.2604041181676, 1369.2604041181676, 1369.2604041181676, 1369.2604041181676, 1369.2604041181676, 1369.2604041181676, 1369.2604041181676, 1369.2604041181676, 1369.2604041181676, 1369.2604041181676, 1369.2604041181676, 1369.2604041181676, 1369.2604041181676


~~~~~~~~~~







PCA and SVD with libraries

In [24]:
from sklearn.decomposition import TruncatedSVD, PCA
from sklearn.preprocessing import StandardScaler
import numpy as np

# Standardize the data
scaler = StandardScaler()
data_standardized = scaler.fit_transform(data)

# Singular Value Decomposition (SVD)
svd = TruncatedSVD(n_components=2) 
svd_result = svd.fit_transform(data_standardized)

# Principal Component Analysis (PCA)
pca = PCA(n_components=2) 
pca_result = pca.fit_transform(data_standardized)


print("SVD Result:")
print(svd_result)

print("\nPCA Result:")
print(pca_result)


SVD Result:
[[-7.79187121e+00  6.06340936e+00]
 [-6.99523483e+00 -2.03990680e-01]
 [-6.88196318e+00  6.87515397e-01]
 [-6.36920681e+00 -2.44936588e-01]
 [-6.54505904e+00 -5.89028204e+00]
 [-6.54395891e+00 -5.87985560e+00]
 [-5.98877799e+00 -4.60405563e-01]
 [-7.97606558e+00  6.32761873e+00]
 [-5.73473504e+00 -4.33544312e-01]
 [-5.86670199e+00 -1.57369561e-01]
 [-7.78141993e+00  6.16246051e+00]
 [-5.66103379e+00  1.23196649e-01]
 [-7.87550582e+00  6.27571740e+00]
 [-5.19628769e+00 -1.17630180e-01]
 [-5.45725159e+00  6.03583430e-01]
 [-4.58625370e+00  2.42108957e-01]
 [-5.42826209e+00  1.59235299e-02]
 [-7.81040191e+00  5.79207166e+00]
 [-5.14769214e+00 -2.48253326e-01]
 [-4.90486945e+00 -3.48357306e-01]
 [-7.93837199e+00  6.24025621e+00]
 [-4.49219116e+00 -2.60662094e-01]
 [-3.64985654e+00 -4.34451647e-01]
 [-4.14497208e+00 -1.90192573e-01]
 [-4.37055899e+00  3.91409625e-01]
 [-3.80782876e+00 -4.51805304e-01]
 [-4.10076270e+00 -4.46170702e-01]
 [-3.79828209e+00 -4.90552209e-01]
 [-6.501

PCA Implementation:
The results obtained from the code I made from scratch differ greatly from those by sklearn. These differences may be because of variations of the algorithms, functions, optimization techniques, or the data preprocessing steps. 
One line of code contributing to the difference may be:
``` Transform the original dataset using the top k eigenvectors 
    X_pca = [[sum(x_i * eigenvector_i for x_i, eigenvector_i in zip(x, top_k_eigenvector)) 
              for x, top_k_eigenvector in zip(X, top_k_eigenvectors)] for top_k_eigenvector in top_k_eigenvectors]
```
SVD Implementation:
Due to the complexity of the task, my SVD implementation faced execution issues, in that after an hour of execution, it has not finished executing, which prevents me from doing a complete evaluation and conclusion of my code's performance. This further shows that using libraries is much more efficient. 
The part of the code which takes up so much time is most likely this:
```
ata = [[sum(a * b for a, b in zip(row_a, col_b)) for col_b in zip(*matrix)] for row_a in matrix]
aat = [[sum(a * b for a, b in zip(row_a, row_b)) for row_b in matrix] for row_a in matrix]
```
The reason is that the nested loops iterate through each element of the matrix twice, which can lead to a significant increase in execution time for larger matrices.